# BI Exercises

## Set Up

In [1]:
%load_ext sql

In [2]:
%config SqlMagic.autopandas=True

In [3]:
%config SqlMagic.displaycon=False

C:\Users\SR133EX\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3106: UserWarning: Config option `displaycon` not recognized by `SqlMagic`.  Did you mean `displaylimit`?
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
%sql sqlite:///transactions.db

'Connected: @transactions.db'

In [5]:
%%sql
SELECT *
FROM transactions
LIMIT 5

 * sqlite:///transactions.db
Done.


,transaction_id,payment_method,seller_id,buyer_id,transaction_date,product_price
0,1294738,1,5107765,5007941,2022-05-11 15:46:38,1410.0
1,585369,4,4205200,4250752,2021-03-27 15:01:28,800.0
2,882933,4,581926,4695227,2021-10-20 14:45:11,800.0
3,1145325,3,536932,4865235,2022-03-06 17:37:04,1616.0
4,762483,5,432561,4186489,2021-08-06 11:33:12,840.0


In [6]:
%%sql
SELECT *
FROM users
LIMIT 5

 * sqlite:///transactions.db
Done.


,user_id,register_date
0,5042412,2022-03-01 21:26:27
1,707930,2020-09-29 08:33:26
2,5283835,2022-06-23 11:18:09
3,4195678,2021-02-24 22:12:15
4,5370530,2022-08-29 13:53:33


## Preguntas

### Agregaciones simples

#### Qué procentaje de usuarias registradas han comprado al menos un producto?

In [7]:
%%sql 
buyer_transactions <<
select user_id, count(distinct transaction_id) as transactions
from users u
left join transactions t
on u.user_id = t.buyer_id
group by user_id

 * sqlite:///transactions.db
Done.
Returning data to local variable buyer_transactions


In [8]:
buyer_transactions['transactions'] = list(map(lambda x: 1 if x > 0 else 0, buyer_transactions['transactions']))
buyer_transactions['transactions'].mean()*100

0.9946666666666666

#### Qué procentaje de compradoras no están registradas?

In [31]:
%%sql 
buyer_registered <<
select user_id, count(distinct register_date) as registered
from transactions t
left join users u
on u.user_id = t.buyer_id
group by buyer_id

 * sqlite:///transactions.db
Done.
Returning data to local variable buyer_registered


In [33]:
buyer_registered

,user_id,registered
0,NaN,0
1,NaN,0
2,NaN,0
3,NaN,0
4,NaN,0
...,...,...
24206,NaN,0
24207,NaN,0
24208,NaN,0
24209,NaN,0


In [32]:
buyer_registered['registered'].mean()

0.03081244062616166

### Retención externa

#### Qué procentaje de usuarias han comprado al menos un producto el mes en el que se registraron?

In [54]:
%%sql
test <<
select 
    user_id,
    strftime('%Y-%m-01', register_date) as register_month,
    strftime('%Y-%m-01', transaction_date) as transaction_month,
    round((julianday(strftime('%Y-%m-01, transaction_date')) - julianday(strftime('%Y-%m-01', reister_sate)) / 30 as integer)
    
from users u
left join transactions t 
on u.user_id = t.buyer_id
where transaction_id is not null

 * sqlite:///transactions.db
(sqlite3.OperationalError) near "as": syntax error
[SQL: select user_id, strftime('%Y-%m-01', register_date) as register_month, strftime('%Y-%m-01', transaction_date) as transaction_month, round((julianday(strftime('%Y-%m-01, transaction_date')) - julianday(strftime('%Y-%m-01', reister_sate)) / 30 as integer) from users u left join transactions t on u.user_id = t.buyer_id where transaction_id is not null]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


In [55]:
retention.groupby(['register_month', 'datediff'], dropna=False)['transaction_month'].count()

NameError: name 'retention' is not defined

#### Cual fue el mes en el que las usuarias que se registraron convirtieron mejor a sellers? (Vendieron más items). Se mantuvo esta tendencia?

### Retención interna

#### Ha habido algún mes en el que las usuarias que compraron ese mes no volvieron a comprar el mes siguiente en mayor medida que el resto?

#### Que porcentaje de las compras cada mes están realizadas por usuarias que nunca han comprado? Y por usuarias con más de 3 meses desde su primera compra?